In [1]:
# -*- coding: utf-8 -*-
import signal
import requests
import time
import numpy as np
import pandas as pd
import _thread

In [2]:
API_KEY = {'X-API-key': 'DIVV75B4'}
shutdown = False

In [3]:
#%% General functions

# This class allows to print error messages
class ApiException(Exception):
    pass

In [4]:
# Class allows gracefull shutdown
def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

In [5]:
#%% Basic information functions

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error, check key.')

def get_security_information(session):
    resp = session.get('http://localhost:9999/v1/securities')
    if resp.ok:
        securities = pd.DataFrame(resp.json())
        return securities
    raise ApiException('Authorization error, check key.')

    
def ticker_bid_ask(session, ticker, quantity=1):
    """
    Get aggregated bid and ask for quantity
    """
    quantity = round(quantity*1.5)
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params=payload)
    if resp.ok:
        book = resp.json()
        # First determine bid
        bid_levels = pd.Series([book['bids'][l]['price'] for l in range(len(book['bids']))])
        bid_volumes = pd.Series([book['bids'][l]['quantity'] for l in range(len(book['bids']))])
        agg_bid_volumes = bid_volumes.sum()
        if quantity > agg_bid_volumes:
            bid = 0
        else:
            q = quantity
            vol_at_levels = pd.Series(dtype=int, index = bid_volumes.index)
            for i in vol_at_levels.index:
                vol_at_levels[i] = min(q, bid_volumes[i])
                q -= min(q, bid_volumes[i])
            bid = (vol_at_levels * bid_levels).sum() / quantity
        
        # Now ask
        # First determine bid
        ask_levels = pd.Series([book['asks'][l]['price'] for l in range(len(book['asks']))])
        ask_volumes = pd.Series([book['asks'][l]['quantity'] for l in range(len(book['asks']))])
        agg_ask_volumes = ask_volumes.sum()
        if quantity > agg_ask_volumes:
            ask = 100000000
        else:
            q = quantity
            vol_at_levels = pd.Series(dtype=int, index = ask_volumes.index)
            for i in vol_at_levels.index:
                vol_at_levels[i] = min(q, ask_volumes[i])
                q -= min(q, ask_volumes[i])
            ask = (vol_at_levels * ask_levels).sum() / quantity
        
        return bid, ask
    raise ApiException('Authorization error, check key.')

In [6]:
#%% Basic trading functions
def buy_market(session, ticker, quantity):
    df = get_security_information(session)
    max_trade_size = df.loc[df.ticker == ticker, 'max_trade_size'].values[0]
    max_trade_size /= 1
    if quantity > max_trade_size:
        while quantity > max_trade_size:
            trade_size = max_trade_size
            payload = {'ticker': ticker, 'type': 'MARKET', 'quantity':int(trade_size), 'action':'BUY'}
            session.post('http://localhost:9999/v1/orders', params=payload)
            quantity -= trade_size
            time.sleep(0.2)
    payload = {'ticker': ticker, 'type': 'MARKET', 'quantity':int(quantity), 'action':'BUY'}
    resp = session.post('http://localhost:9999/v1/orders', params=payload)
    return resp 

def sell_market(session, ticker, quantity):
    df = get_security_information(session)
    max_trade_size = df.loc[df.ticker == ticker, 'max_trade_size'].values[0]
    max_trade_size /= 1
    if quantity > max_trade_size:
        while quantity > max_trade_size:
            trade_size = max_trade_size
            payload = {'ticker': ticker, 'type': 'MARKET', 'quantity':int(trade_size), 'action':'SELL'}
            session.post('http://localhost:9999/v1/orders', params=payload)
            quantity -= trade_size
            time.sleep(0.2)
    payload = {'ticker': ticker, 'type': 'MARKET', 'quantity':int(quantity), 'action':'SELL'}
    resp = session.post('http://localhost:9999/v1/orders', params=payload)
    return resp

In [7]:
def lbuy(session, ticker, quantity,price):
    
    payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':quantity, 'action':'BUY','price':price}
    resp = session.post('http://localhost:9999/v1/orders', params=payload)
    return resp
def lsell(session, ticker, quantity, price):

    payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':quantity, 'action':'SELL','price':price}
    session.post('http://localhost:9999/v1/orders', params=payload)  

In [8]:
def get_position(ticker):
    with requests.Session() as s:
        s.headers.update(API_KEY)  
        ticker_info=s.get('http://localhost:9999/v1/securities',params ={'key':'DIVV75B4','ticker':ticker}).json()
        #print('ticker:{} | position: {}'.format( ticker, ticker_info[0]['position']))
        position=int((ticker_info[0]['position'])) 
        return position

def closeposition(ticker):
    with requests.Session() as s:
        s.headers.update(API_KEY)  
        p=get_position(ticker)
        if p<0:
            buy_market(s,ticker,np.abs(p))
        elif p>0:
            sell_market(s,ticker,np.abs(p))


def closeallpos():
    closeposition('RITC')
    closeposition('USD')
    closeposition('BULL')
    closeposition('BEAR')
 

## New limit functions

In [9]:
def lbuy_1(session, ticker, quantity, price):
    
    df = get_security_information(session)
    max_trade_size = df.loc[df.ticker == ticker, 'max_trade_size'].values[0]
    max_trade_size /= 1
    
    if quantity > max_trade_size:
        while quantity > max_trade_size:
            trade_size = max_trade_size
            payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':int(trade_size), 'action':'BUY', 'price':price}
            session.post('http://localhost:9999/v1/orders', params=payload)
            quantity -= trade_size
            
    payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':int(quantity), 'action':'BUY','price':price}
    resp = session.post('http://localhost:9999/v1/orders', params=payload)
    return resp

def lsell_1(session, ticker, quantity, price):
    
    df = get_security_information(session)
    max_trade_size = df.loc[df.ticker == ticker, 'max_trade_size'].values[0]
    max_trade_size /= 1
    
    if quantity > max_trade_size:
        while quantity > max_trade_size:
            trade_size = max_trade_size
            payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':int(trade_size), 'action':'SELL', 'price':price}
            session.post('http://localhost:9999/v1/orders', params=payload)
            quantity -= trade_size
            
    payload = {'ticker': ticker, 'type': 'LIMIT', 'quantity':int(quantity), 'action':'SELL','price':price}
    resp = session.post('http://localhost:9999/v1/orders', params=payload)  
    return resp

## New kill orders functions

In [10]:
def kill_one_order(session, order_id):
    """
    If know order id and want to cancel 1 order
    """
    resp = session.delete(f'http://localhost:9999/v1/orders/{order_id}')
    return resp

In [11]:
def kill_orders(session, all_=False, ticker=None, min_price=None, max_price=None):
    """
    if all_ = True cancel all orders
    if ticker is specified then cancel all orders for this ticker
    it min_price or max_price is specified then cancel all orders with higher or lower price
    Possible to implement only buy or sell orders cancel
    Gives 400 error when using all_=True but works 
    """
    if all_:
        resp = session.post('http://localhost:9999/v1/commands/cancel', 
                    params={'all':1})
    elif ticker:
        if min_price:
            resp = session.post('http://localhost:9999/v1/commands/cancel', 
                    params={'query': 'Ticker=' + "'" + ticker + "'" +" AND " + f"Price>{min_price}"})
        elif max_price:
            resp = session.post('http://localhost:9999/v1/commands/cancel', 
                    params={'query': 'Ticker=' + "'" + ticker + "'" +" AND " + f"Price<{max_price}"})
        else:
            resp = session.post('http://localhost:9999/v1/commands/cancel', 
                    params={'ticker': ticker})
            
    return resp

## Main functions

In [12]:
def num_format(number):
    return ','.join(str(number).split('.'))

In [13]:
#%% Tenders    

def accept_tenders(session):
    change = True
    resp = session.get('http://localhost:9999/v1/tenders')
    if resp.ok:
        tender = pd.DataFrame(resp.json())
        if len(tender.index) > 0:
            for i in tender.index:
                
                tender_id = tender.loc[i, 'tender_id']
                ticker = tender.loc[i, 'ticker']
                quantity = int(tender.loc[i, 'quantity'])
                tprice = tender.loc[i, 'price']
                action = tender.loc[i, 'action']
                fixed_bid =  tender.loc[i, 'action']
                
                df = get_security_information(session) 
                lastPrice= df.loc[df.ticker == ticker, 'last'].values[0]
                lastBid= df.loc[df.ticker == ticker, 'bid'].values[0]
                lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]
                
                min_tender_profit=0.003
                
                if fixed_bid:
                    if action == 'BUY':
                        if tprice * (1 + min_tender_profit) < lastPrice:
                            session.post('http://localhost:9999/v1/tenders/' + str(tender_id))
                            
                            price=tprice*1.002
                                
                            q = abs(quantity)
                            
                            lsell_1(session, 'RITC', quantity, price)
                            
                            time.sleep(0.2)
                            
                            if (lastBid-tprice)/tprice>-0.002:
                                
                                while q > 0 and (lastBid-tprice)/tprice>-0.002:  
                                    
                                    price=lastBid-0.001
                                       
                                    quantity=q
                                    lsell_1(session, ticker, quantity, price)
                                    time.sleep(0.2)
                                    
                                    q = abs(get_position(ticker))
                                    df = get_security_information(session)
                                    lastBid = df.loc[df.ticker == ticker, 'bid'].values[0]
                                    
                                kill_orders(session, ticker=ticker)
                                closeposition(ticker)
                                
                            else:
                                
                                kill_orders(session, ticker=ticker)
                                closeposition(ticker)
                                
                    else:
                         if tprice / (1 + min_tender_profit) > lastPrice:
                            session.post('http://localhost:9999/v1/tenders/' + str(tender_id))
                            price=tprice*(1-0.002)
                            q = abs(quantity)
                            
                            lbuy_1(session, 'RITC', quantity,price)
                            
                            time.sleep(0.2)
                            
                            if (-lastAsk + tprice)/tprice>-0.002:
                                while q > 0 and (-lastAsk + tprice)/tprice>-0.002:    
                                    
                                    price = lastAsk + 0.001
                                        
                                    quantity = q
                                    lbuy_1(session, ticker, quantity, price)
                                    time.sleep(0.2)
                                    
                                    q = abs(get_position(ticker))
                                    df = get_security_information(session)
                                    lastAsk = df.loc[df.ticker == ticker, 'ask'].values[0]
                                    
                                kill_orders(session, ticker=ticker)
                                closeposition(ticker)

                            else:
                                
                                closeposition(ticker)
                                kill_orders(session, ticker=ticker)

In [14]:
def vol_better(price, bid=True):
    resp = requests.get('http://localhost:9999/v1/securities/book',
                   params = {'key':API_KEY['X-API-key'], 'ticker':'RITC'}).json()
    
    if bid:
        df = pd.DataFrame(resp['bids'])
        df['quantity'] = df['quantity'] - df['quantity_filled']
        return df[df['price'] > price]['quantity'].sum()
    else:
        df = pd.DataFrame(resp['asks'])
        df['quantity'] = df['quantity'] - df['quantity_filled']
        return df[df['price'] < price]['quantity'].sum()

In [18]:
def accept_tenders(session):
    resp = session.get('http://localhost:9999/v1/tenders')
    if resp.ok:
        tender = pd.DataFrame(resp.json())
        if resp:
            if len(tender.index) > 0:
                for i in tender.index:

                    tender_id = tender.loc[i, 'tender_id']
                    ticker = tender.loc[i, 'ticker']
                    quantity = int(tender.loc[i, 'quantity'])
                    tprice = tender.loc[i, 'price']
                    action = tender.loc[i, 'action']
                    fixed_bid =  tender.loc[i, 'action']

                    df = get_security_information(session)
                    lastPrice= df.loc[df.ticker == ticker, 'last'].values[0]
                    lastBid= df.loc[df.ticker == ticker, 'bid'].values[0]
                    lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]

                    min_tender_profit=0.002
                
                    if fixed_bid:
                        if action == 'BUY':
                            if tprice * (1 + min_tender_profit) < lastPrice:

                                session.post('http://localhost:9999/v1/tenders/' + str(tender_id))
                                price=tprice*1.003
                                lsell_1(session, 'RITC', quantity, price)
                                print(f'Ask at {price}')

                                vol = vol_better(price, bid=False)
                                while vol <= 40000:
                                    time.sleep(0.5)
                                    vol = vol_better(price, bid=False)

                                print('Sleeping done')
                                kill_orders(session, ticker=ticker)

                                p = get_position('RITC')
                                print(f'Cur pos: {p}')

                                df = get_security_information(session)
                                lastBid = df.loc[df.ticker == ticker, 'bid'].values[0]
                                lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]

                                if (lastBid - tprice)/tprice >= -0.002 and p > 0:

                                    while p > 0 and (lastBid-tprice)/tprice>=-0.002:

                                        price=lastAsk-0.01

                                        quantity=min([20000,abs(p)])
                                        lsell_1(session, ticker, quantity, price)
                                        time.sleep(1)

                                        p = get_position(ticker)
                                        df = get_security_information(session)
                                        lastBid = df.loc[df.ticker == ticker, 'bid'].values[0]
                                        lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]

                                        print('Doing limit')


                                    kill_orders(session, ticker=ticker)
                                    closeposition(ticker)
                                    print('Closing market')

                                elif p!=0 and (lastBid-tprice)/tprice<-0.002:

                                    kill_orders(session, ticker=ticker)
                                    closeposition(ticker)
                                    print('Closing market')

                        else:
                             if tprice * (1 - min_tender_profit) > lastPrice:

                                session.post('http://localhost:9999/v1/tenders/' + str(tender_id))
                                price=tprice*(1-0.003)

                                lbuy_1(session, 'RITC', quantity,price)
                                vol = vol_better(price, bid=False)

                                while vol <= 40000:
                                    time.sleep(0.5)
                                    vol = vol_better(price, bid=True)
                                print('Sleeping done')

                                kill_orders(session, ticker=ticker)
                                p=get_position('RITC')
                                print(f'Cur pos: {p}')

                                df = get_security_information(session)
                                lastBid = df.loc[df.ticker == ticker, 'bid'].values[0]
                                lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]

                                if (-lastAsk + tprice)/tprice>=-0.002 and p < 0:
                                    while p < 0 and (-lastAsk + tprice)/tprice>=-0.002:
                                        print('Doing limit')

                                        price = lastBid+0.01 

                                        quantity = min([20000, abs(p)])
                                        lbuy_1(session, ticker, quantity, price)
                                        time.sleep(1)

                                        p = get_position(ticker)
                                        df = get_security_information(session)
                                        lastBid = df.loc[df.ticker == ticker, 'bid'].values[0]
                                        lastAsk=df.loc[df.ticker == ticker, 'ask'].values[0]

                                    kill_orders(session, ticker=ticker)
                                    closeposition(ticker)
                                    print('Closing market')

                                elif (-lastAsk + tprice)/tprice<-0.002 and p!=0:

                                    closeposition(ticker)
                                    kill_orders(session, ticker=ticker)
                                    print('Closing market')
        else:
            pass

In [16]:
def main():
    with requests.Session() as s:
        s.headers.update(API_KEY)
        while True:
            
            tick = get_tick(s)
            if tick > 0 and tick < 297:
#                 print(tick)
                accept_tenders(s)
                case_securities = get_security_information(s)
                usd = int(case_securities.loc[case_securities.ticker == 'USD', 'position'].values[0])
                if usd > 0:
                    sell_market(s, 'USD', usd)
                if usd < 0:
                    buy_market(s, 'USD', np.abs(usd))
                
               
                tick = get_tick(s)
            time.sleep(0.5)
        